# Basic TensorFlow

About

Intro

In [1]:
import io
import os
import pickle
from urllib.request import urlretrieve

from zipfile import ZipFile

import numpy as np
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm

print('All modules imported.')

All modules imported.


Download the notMNIST training and test dataset

In [2]:
def download(url, file):
    """
    Download file from <url>
    """
    if not os.path.isfile(file):
        print('Downlading ' + file + '...')
        urlretrieve(url, file)
        print('Download Finished')

# ToDo: Add URL
# Download the training and test dataset
download('', 'notMNIST_train.zip')
download('', 'notMNIST_test.zip')

print('All files downloaded.')

All files downloaded.


Get the features and labels from the zip files

In [3]:
def uncompress_features_labels(file):
    """
    Uncompress features and labels from zip file
    """
    features = []
    labels = []

    with ZipFile(file) as zipf:
        filenames_pbar = tqdm(zipf.namelist(), unit='files')
        for filename in filenames_pbar:
            # Check if the file is a directory
            if not filename.endswith('/'):
                with zipf.open(filename) as image_file:
                    image = Image.open(image_file)
                    image.load()
                    # Load image data as 1 dimensional array
                    feature = np.array(image).flatten()

                # Get the the letter from the filename
                label = os.path.split(filename)[1][0]

                features.append(feature)
                labels.append(label)
    return np.array(features), np.array(labels)

# Get the features and labels from the zip files
train_features, train_labels = uncompress_features_labels('notMNIST_train.zip')
test_features, test_labels = uncompress_features_labels('notMNIST_test.zip')

print('All features and labels uncompressed.')

100%|██████████| 10001/10001 [00:01<00:00, 6392.77files/s]

All features and labels uncompressed.


In [8]:
# Normalize the features
# Apply zero mean and zero variance scale to the image features
def normalize_data(data):
    #  ToDo: Problem 1 - Implement function to normalize data

train_features = normalize_data(train_features)
test_features = normalize_data(test_features)

# Test Cases
np.testing.assert_array_almost_equal(
    normalize_data(np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])),
    np.array([-0.4, -0.3, -0.2, -0.099, 0.0, 0.099, 0.199, 0.3, 0.4, 0.5]),
    decimal=3)
np.testing.assert_array_almost_equal(
    normalize_data(np.array([-100, -30, -1000, -20, -20, -10, -10, -20, -10, -10])),
    np.array([9.5, 2.5, 99.5, 1.5, 1.5, 0.5, 0.5, 1.5, 0.5, 0.5]))

print('Tests Passed!')

Tests Passed!


In [9]:
# Get randomized datasets for training and validation
train_features, valid_features, train_labels, valid_labels = train_test_split(
    train_features,
    train_labels,
    test_size=0.05,
    random_state=832289)

print('Training features and labels randomized and split.')

Training features and labels randomized and split.


In [8]:
# Save the data for easy access
print('Saving data to pickle file...')
pickle_file = 'notMNIST.pickle'
if not os.path.isfile(pickle_file):
    try:
        with open('notMNIST.pickle', 'wb') as pfile:
            pickle.dump(
                {
                    'train_dataset': train_features,
                    'train_labels': train_labels,
                    'valid_dataset': valid_features,
                    'valid_labels': valid_labels,
                    'test_dataset': test_features,
                    'test_labels': test_labels,
                },
                pfile, pickle.HIGHEST_PROTOCOL)
    except Exception as e:
        print('Unable to save data to', pickle_file, ':', e)
        raise

print('Data cached in pickle file.')

Saving data to pickle file...
File saved


In [17]:
# Ask the student to get a bad accuracy and a good accuracy by tweaking the learning rate and number of steps
learning_rate = 0.5
num_steps = 801

# The input is the images of letters. The images are 28 by 28 pixels. The image has a total size of 784. 28*28=784
num_features = 28*28
# The labels are A-J, which make a total of 10 classes
num_labels = 10

# Limit the amount of training data to use in the neural network.
# Using all the training data will take too long to process.
train_subset = 10000
if len(train_features) > train_subset:
    train_features = train_features[:train_subset, :]
    train_labels = train_labels[:train_subset, :]

features = tf.placeholder(tf.float32, shape=(None, num_features))
labels = tf.placeholder(tf.float32, shape=(None, num_labels))

weights = tf.Variable(tf.truncated_normal([num_features, num_labels]))
biases = tf.Variable(tf.zeros([num_labels]))

# Linear Regression Function WX + b
logits = tf.matmul(features, weights) + biases

prediction = tf.nn.softmax(logits)

# Cross entropy
cross_entropy = -tf.reduce_sum(labels * tf.log(prediction), reduction_indices=1)

# Training loss
loss = tf.reduce_mean(cross_entropy)

# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

# Determine if the predictions are correct
is_correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(labels, 1))
# Calculate the accuracy of the predictions
accuracy = tf.reduce_mean(tf.cast(is_correct_prediction, tf.float32))

# Create an operation that initializes all variables
init = tf.initialize_all_variables()

with tf.Session() as session:
    session.run(init)

    # The training cycle
    for step in range(num_steps):
        # Run optimizer and get loss
        _, l = session.run(
            [optimizer, loss],
            feed_dict={
                features: train_features,
                labels: train_labels})

        # Display every 100 epochs
        if not step % 100:
            training_accuracy = session.run(
                    accuracy,
                    feed_dict={
                        features: train_features,
                        labels: train_labels})
            validation_accuracy = session.run(
                    accuracy,
                    feed_dict={
                        features: valid_features,
                        labels: valid_labels})
            print('Step: {:>3}  Loss: {:>20}  Train Acc: {:>5}  Vald Acc: {:>20}'.format(
                step,
                l,
                training_accuracy,
                validation_accuracy))

    # Check accuracy against Test data
    test_accuracy = session.run(
        accuracy,
        feed_dict={
            features: test_features,
            labels: test_labels})
    print('Test Acc: {}'.format(test_accuracy))

assert test_accuracy >= 0.80

print('Nice Job!')

Step:   0  Loss:   16.557613372802734  Train Acc: 0.11180000007152557  Vald Acc:  0.10819047689437866
Step: 100  Loss:    2.340564727783203  Train Acc: 0.7185999751091003  Vald Acc:   0.7080000042915344
Step: 200  Loss:   1.8992466926574707  Train Acc: 0.7479000091552734  Vald Acc:   0.7286666631698608
Step: 300  Loss:   1.6496416330337524  Train Acc: 0.7591999769210815  Vald Acc:   0.7366666793823242
Step: 400  Loss:   1.4782439470291138  Train Acc: 0.7694000005722046  Vald Acc:   0.7405714392662048
Step: 500  Loss:   1.3507778644561768  Train Acc: 0.7771000266075134  Vald Acc:   0.7442857027053833
Step: 600  Loss:   1.2511628866195679  Train Acc: 0.78329998254776  Vald Acc:   0.7463809251785278
Step: 700  Loss:   1.1708112955093384  Train Acc: 0.7903000116348267  Vald Acc:    0.748285710811615
Step: 800  Loss:   1.1044509410858154  Train Acc: 0.7946000099182129  Vald Acc:   0.7492380738258362
Test Acc: 0.8242999911308289
Nice Job!
